In [1]:
from IPython.display import HTML
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, WebDriverException
import requests
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import random

In [ ]:
# Lấy về danh sách các proxy IP và port từ https://sslproxies.org
def get_proxies():
    url = 'https://sslproxies.org/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    proxies = []
    count = 0
    for row in soup.find_all('tr')[1:]:
        # Chỉ lấy khoảng 20 proxies
        if(count>=20):
            break
        columns = row.find_all('td')
        # phải có ít nhất hai elements thì mới lấy
        if columns[4].text == "elite proxy" and columns[5].text == "yes":
            ip = columns[0].text
            port = columns[1].text
            proxies.append(f'{ip}:{port}')
        count +=1
    return proxies

def get_vietnamese_proxies():
    url = 'http://free-proxy.cz/en/proxylist/country/VN/all/ping/all'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    proxies = []
    count = 0
    # Tìm table có id 'proxy_list'
    proxy_table = soup.find('table', {'id': 'proxy_list'})
    if proxy_table:
        # Find all rows in the table
        rows = proxy_table.find_all('tr')
        for row in rows[1:]:
            # Get all cells in the current row
            cells = row.find_all(['th', 'td'])
            # Loop through each cell (column) in the row
            for index, cell in enumerate(cells):
                if index in [0, 1]:
                    print(cell.text)
    return proxies

# lấy ra danh sách các proxies
proxy_list = get_proxies()

# lấy ra danh sách các proxies của Việt Nam
vn_proxy_list = get_vietnamese_proxies()

# Chọn một proxy ngẫu nhiên
random_proxy = random.choice(proxy_list)
print('Proxy: ' + random_proxy)

In [2]:
def get_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument(f'--proxy-server={vn_random_proxy}')
    driver = webdriver.Chrome(options=chrome_options, service=ChromeService(ChromeDriverManager().install()))
    return driver

# AUTO VIEW TRUYENHD

In [3]:
# Hàm auto scroll, refresh sau nhiều lần và click next chương bằng Selenium
def open_and_refresh_websiteVL(driver, url, interval_seconds, number_of_chapter = 0):
    try:
        driver.get(url)
    except ConnectionRefusedError:
        print(f'WebDriverException caught at {number_of_chapter}, url: {url}')
        driver.quit()
        new_driver = get_chrome_driver()
        open_and_refresh_websiteVL(new_driver, url, interval_seconds, number_of_chapter)
    except WebDriverException:
        driver.quit()
        return
    count = 0
    while(count<=2):
        for _ in range(5):
            # Dùng JavaScript để từ từ cuộn xuống
            driver.execute_script("window.scrollBy(0, 220);")
            time.sleep(1)
            driver.execute_script("window.scrollBy(0, 220);")
        driver.refresh()
        time.sleep(interval_seconds)
        count+=1
    if(number_of_chapter > 80):
        driver.quit()
    if(driver.find_elements(By.XPATH, '//*[@id="html"]/body/div[5]/div/div/div[8]/div[3]/a')!=[]): 
        buttonNext = driver.find_element(By.XPATH, '//*[@id="html"]/body/div[5]/div/div/div[8]/div[3]/a').click()
    nexturl = driver.current_url
    open_and_refresh_websiteVL(driver,nexturl, interval_seconds, number_of_chapter+1)
    
    # Close the WebDriver when the notebook cell is done
    # This is optional and depends on whether you want to keep the browser open
    # If you want to keep the browser open, omit the next line

# Set the website URL
website_url = "https://truyenhdx.com/truyen/the-authors-pov-goc-nhin-cua-tac-gia/chap/8393100-chuong-2/"
website_url_HD50 = "https://truyenhdx.com/truyen/the-authors-pov-goc-nhin-cua-tac-gia/chap/8393100-chuong-50/"

url_TKLLPG_1 = 'https://truyenhdx.com/truyen/than-kiem-luu-lac-phap-gioi/chap/10323547-chuong-1/'


# Số giây cách
refresh_interval = 3
d = get_chrome_driver()
# Gọi đến function auto refresh và auto click next chương
open_and_refresh_websiteVL(d,website_url, refresh_interval)